In [1]:
import os
import re
import sys
import pandas as pd
sys.path.insert(0, './_modules')
from db_functions import TheDB
#from logging_module import getlogger
from logging_module_v2 import StartLogger
from regex_module import FindEmail, FindNumbersBraket, FindLawsuit, FindCaseId

In [2]:
## bots BD connections
conn_dict = {'host':'localhost', 'database': 'brspcqadev',
             'user':'root', 'password':'quito2020'}

cqdb = TheDB(conn_dict)

In [3]:
data_path = os.path.join(os.getcwd(), '_data_output')

In [4]:
os.listdir(data_path)

['tmail_data_clean_bigrams_cluster_3_2020-03-01.csv',
 'tmail_data_clean_bigrams_cluster_22_2020-03-01.csv',
 'tmail_data_clean_bigrams_cluster_12_2020-03-01.csv',
 'tmail_data_clean_bigrams_cluster_9_2020-04-01.csv',
 'mail_data_clean_bigrams.csv',
 'tmail_data_clean_bigrams_cluster_16_2020-03-01.csv',
 'tmail_data_clean_bigrams_cluster_4_2020-03-01.csv',
 'tmail_data_clean_bigrams_cluster_9_2020-03-10.csv',
 'tmail_data_clean_bigrams_cluster_18_2020-03-01.csv',
 'tmail_data_clean_bigrams_cluster_15_2020-04-05.csv',
 'tmail_data_clean_bigrams_cluster_9_2020-03-04.csv',
 'tmail_data_clean_bigrams_cluster_20_2020-03-01.csv',
 'tmail_data_clean_bigrams_cluster_9_2020-03-26.csv',
 'tmail_data_clean_bigrams_cluster_8_2020-03-04.csv',
 'MainTrix_BoletosAds.csv',
 'mail_data_clean_bigrams_optimalcluster.csv',
 'tmail_data_clean_bigrams_cluster_2_2020-03-01.csv',
 'mail_data_clean_bigrams_optimalcluster_2020-04-01.csv',
 'tmail_data_clean_bigrams_cluster_8_2020-03-01.csv',
 'oficios_google_cl

In [5]:
df = pd.read_csv(os.path.join(data_path, 'main_trix_clean.csv'), sep = '\t')
print(df.columns)
df = df.rename(columns = {'Case # (DLBCA and Google reference)':'case_id',\
                     'line':'line_id', 'Lawsuit Number':'external_ref_number', 'Deadline':'deadline'})
df = df.loc[:, ['line_id', 'case_id', 'external_ref_number', 'deadline']]
df['external_ref_number'] = df['external_ref_number'].apply(FindLawsuit)
df['case_id'] = df['case_id'].apply(FindCaseId)

Index(['line', 'Case # (DLBCA and Google reference)', 'UID',
       'Date of the Court Orders', 'Court', 'same court', 'Lawsuit Number',
       'CP?', 'Summary of the request/what to produce [internal team rec]',
       'STQ confirm scope of production', 'Deadline',
       'DLBCA's suggested answer', 'Previous Orders and Answers',
       'Google's proposed answer', 'LIS: \nInformatin about case',
       'LIS: \nPolicy and content check',
       'LIS: \nGoogle Inc answer sent to authority? which one?',
       'LIS:\nPriority', 'LIS:\nAssignee', 'Status',
       'Status - writ of mandamus', 'Law Firm'],
      dtype='object')


In [6]:
df

,line_id,case_id,external_ref_number,deadline
0,1,,0009460-19.2013.403.6181,2014-02-28 00:00:00
1,2,6-9080000028545,0002618-91.2011.403.6181,2013-11-28 00:00:00
2,3,9-5821000001559,0007067-24.2013.403.6181,2013-10-22 00:00:00
3,4,5-6191000002205,,2013-11-22 00:00:00
4,5,9-9709000000828,0076690-72.2011.805.0001,2013-10-21 00:00:00
...,...,...,...,...
7614,7740,7-9812000028504,5075998-22.2019.4.04.7000,2019-12-24 00:00:00
7615,7741,6-8938000029130,0330591-87.2019.8.05.0001,2019-12-30 00:00:00
7616,7742,0-5628000028599,505-23.2019.4.01.3601,2020-01-03 00:00:00
7617,7743,0-5628000028599,505-23.2019.4.01.3601,2020-12-24 00:00:00


In [7]:
## input some date, since those fields are manadatory
df['date_received'] = '2020-08-12'
df['date_received_cq'] = '2020-08-12'

In [8]:
df = df[df['line_id'].str.isdigit() == True]
df['line_id'] = df['line_id'].astype(int)

/home/dev/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [9]:
df

,line_id,case_id,external_ref_number,deadline,date_received,date_received_cq
0,1,,0009460-19.2013.403.6181,2014-02-28 00:00:00,2020-08-12,2020-08-12
1,2,6-9080000028545,0002618-91.2011.403.6181,2013-11-28 00:00:00,2020-08-12,2020-08-12
2,3,9-5821000001559,0007067-24.2013.403.6181,2013-10-22 00:00:00,2020-08-12,2020-08-12
3,4,5-6191000002205,,2013-11-22 00:00:00,2020-08-12,2020-08-12
4,5,9-9709000000828,0076690-72.2011.805.0001,2013-10-21 00:00:00,2020-08-12,2020-08-12
...,...,...,...,...,...,...
7614,7740,7-9812000028504,5075998-22.2019.4.04.7000,2019-12-24 00:00:00,2020-08-12,2020-08-12
7615,7741,6-8938000029130,0330591-87.2019.8.05.0001,2019-12-30 00:00:00,2020-08-12,2020-08-12
7616,7742,0-5628000028599,505-23.2019.4.01.3601,2020-01-03 00:00:00,2020-08-12,2020-08-12
7617,7743,0-5628000028599,505-23.2019.4.01.3601,2020-12-24 00:00:00,2020-08-12,2020-08-12


In [10]:
df_db_trix = cqdb.GetTable('Trix')
df_db_trix

,line_id,case_id,external_ref_number,deadline,date_received,date_received_cq,auth_last_name,info_case,case_country,email_response,...,date_status_ms,deadline_ms,ms_info,letter_reviewed,user_in_charge_production,production_completed,production_ready,monitor,emissary_chosen,closure
0,1,None,None,0001-01-01 00:00:00,0001-01-01 00:00:00,2019-06-17 17:22:28,None,None,None,None,...,2020-08-14 21:21:41,2020-08-14 21:21:41,None,None,0,None,None,0,0,None


In [11]:
df['line_id'] = df['line_id'].astype(int) + df_db_trix['line_id'].max()

/home/dev/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [12]:
df = df.drop_duplicates(subset = ['line_id'])

In [13]:
df['date_received'] = pd.to_datetime(df['date_received'])
df['date_received_cq'] = pd.to_datetime(df['date_received_cq'])

/home/dev/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/home/dev/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [14]:
df

,line_id,case_id,external_ref_number,deadline,date_received,date_received_cq
0,2,,0009460-19.2013.403.6181,2014-02-28 00:00:00,2020-08-12,2020-08-12
1,3,6-9080000028545,0002618-91.2011.403.6181,2013-11-28 00:00:00,2020-08-12,2020-08-12
2,4,9-5821000001559,0007067-24.2013.403.6181,2013-10-22 00:00:00,2020-08-12,2020-08-12
3,5,5-6191000002205,,2013-11-22 00:00:00,2020-08-12,2020-08-12
4,6,9-9709000000828,0076690-72.2011.805.0001,2013-10-21 00:00:00,2020-08-12,2020-08-12
...,...,...,...,...,...,...
7614,7741,7-9812000028504,5075998-22.2019.4.04.7000,2019-12-24 00:00:00,2020-08-12,2020-08-12
7615,7742,6-8938000029130,0330591-87.2019.8.05.0001,2019-12-30 00:00:00,2020-08-12,2020-08-12
7616,7743,0-5628000028599,505-23.2019.4.01.3601,2020-01-03 00:00:00,2020-08-12,2020-08-12
7617,7744,0-5628000028599,505-23.2019.4.01.3601,2020-12-24 00:00:00,2020-08-12,2020-08-12


In [15]:
df_db_trix = cqdb.GetTable('Trix')

In [16]:
df_db_trix

,line_id,case_id,external_ref_number,deadline,date_received,date_received_cq,auth_last_name,info_case,case_country,email_response,...,date_status_ms,deadline_ms,ms_info,letter_reviewed,user_in_charge_production,production_completed,production_ready,monitor,emissary_chosen,closure
0,1,None,None,0001-01-01 00:00:00,0001-01-01 00:00:00,2019-06-17 17:22:28,None,None,None,None,...,2020-08-14 21:21:41,2020-08-14 21:21:41,None,None,0,None,None,0,0,None


In [17]:
type(df_db_trix['line_id'][0])

numpy.int64

In [18]:
type(df['line_id'][0])

numpy.int64

In [19]:
cqdb.TablesDataType('Trix')

,0,1,2,3,4,5
0,line_id,int(11),NO,PRI,None,auto_increment
1,case_id,longtext,YES,,None,
2,external_ref_number,longtext,YES,,None,
3,deadline,datetime,YES,,None,
4,date_received,datetime(6),NO,,None,
5,date_received_cq,datetime(6),NO,,None,
6,auth_last_name,longtext,YES,,None,
7,info_case,longtext,YES,,None,
8,case_country,longtext,YES,,None,
9,email_response,longtext,YES,,None,


In [20]:
df_db_trix.T

,0
line_id,1
case_id,None
external_ref_number,None
deadline,0001-01-01 00:00:00
date_received,0001-01-01 00:00:00
date_received_cq,2019-06-17 17:22:28
auth_last_name,None
info_case,None
case_country,None
email_response,None


In [22]:
df_db_trix['deadline'][0]

datetime.datetime(1, 1, 1, 0, 0)

In [24]:
type(df_db_trix['auth_last_name'][0])

NoneType

In [25]:
df_test = df[:1].drop('line_id', axis=1)
df_test

,case_id,external_ref_number,deadline,date_received,date_received_cq
0,,0009460-19.2013.403.6181,2014-02-28 00:00:00,2020-08-12,2020-08-12


In [26]:
df_model = df_db_trix.copy()

In [27]:
df_model

,line_id,case_id,external_ref_number,deadline,date_received,date_received_cq,auth_last_name,info_case,case_country,email_response,...,date_status_ms,deadline_ms,ms_info,letter_reviewed,user_in_charge_production,production_completed,production_ready,monitor,emissary_chosen,closure
0,1,None,None,0001-01-01 00:00:00,0001-01-01 00:00:00,2019-06-17 17:22:28,None,None,None,None,...,2020-08-14 21:21:41,2020-08-14 21:21:41,None,None,0,None,None,0,0,None


In [34]:
columns_list = list(df_model.columns)

In [36]:
df_final = pd.concat([df_model, df_test], sort = False).loc[:, columns_list]

In [43]:
df_test.merge(df_model)

ValueError: You are trying to merge on datetime64[ns] and object columns. If you wish to proceed you should use pd.concat

In [40]:
df_final = df_final.drop('line_id', axis = 1)

In [41]:
df_final

,case_id,external_ref_number,deadline,date_received,date_received_cq,auth_last_name,info_case,case_country,email_response,status,...,date_status_ms,deadline_ms,ms_info,letter_reviewed,user_in_charge_production,production_completed,production_ready,monitor,emissary_chosen,closure
0,None,None,0001-01-01 00:00:00,0001-01-01 00:00:00,2019-06-17 17:22:28,None,None,None,None,Waiting Advice,...,2020-08-14 21:21:41,2020-08-14 21:21:41,None,None,0.0,None,None,0.0,0.0,None
0,,0009460-19.2013.403.6181,2014-02-28 00:00:00,2020-08-12 00:00:00,2020-08-12 00:00:00,NaN,NaN,NaN,NaN,NaN,...,NaT,NaT,None,None,NaN,None,None,NaN,NaN,None


In [ ]:
df_test['label'] = 1
df_test['date_created'] = '2020-08-13'
df_test['date_modified'] = '2020-08-13'
df_test['users_id'] = 1
df_test['user_advice_correction'] = 1
df_test['advice_approved'] = 0
df_test['advice_sent'] = 0
df_test['petition_cat_id'] = 1
df_test['date_status_ms'] =  '2020-08-13'
df_test['deadline_ms'] =  '2020-08-13'
df_test['user_in_charge_production'] = 1
df_test['monitor'] = 1
df_test['emissary_chosen'] = 1

In [ ]:
df_test['date_created'] = pd.to_datetime(df_test['date_created'])
df_test['date_modified'] = pd.to_datetime(df_test['date_modified'])
df_test['date_status_ms'] = pd.to_datetime(df_test['date_status_ms'])
df_test['deadline_ms'] = pd.to_datetime(df_test['deadline_ms'])

In [ ]:
df_test

In [ ]:
df_db_trix2 = cqdb.InsertDB(df_test,\
                            'Trix', key_column = '' , return_table = True, append_table = True)

In [ ]:
cqdb.GetTable('Trix')